# Outline

- topic modelling by using **Latent Dirichlet Allocation**
- train data is `matichon.json` that contains about 16K articles of Thai news by Matichon between 2015-2018
- filter only `politics` category 

# Import

In [67]:
import pandas as pd
import re

from gensim.models import LdaModel  # LDA
from gensim.corpora.dictionary import Dictionary  # for count words 

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

# Load data

- `headline` and `article` are untokenized text
- filter by `category` == `politics`

In [68]:
df = pd.read_json('data/matichon.json')
df

,headline,article,date,category,url,id
0,เปิดใจชัดๆ กับ'ชูวิทย์' สบู่ ขัน ต้องเตรียมไว้...,นายชูวิทย์ กมลวิศิษฎ์ หัวหน้าพรรครักประเทศไทย ...,2015-08-16 10:58:27,politics,https://www.matichon.co.th/politics/news_104,104
1,ลุ้นบาททะลุ40ต่อดอลล์ ปลายปี'59 เหตุสงครามค่าเ...,แบงก์ซีไอเอ็มบี ชี้การฟื้นฟูเศรษฐกิจติดปัญหารา...,2015-08-26 11:19:03,economy,https://www.matichon.co.th/economy/news_148,148
2,"ศาลสูงแอฟริกาใต้ตัดสินให้ ""พิสตอเรียสผิดฐานฆาต...",จากรายงานของบีบีซี ออสการ์ พิสตอเรียส นักวิ่งร...,2015-12-04 02:52:25,foreign,https://www.matichon.co.th/foreign/news_247,247
3,แม่แดนผู้ดีฮึด! อุ้มท้องลูกแฝดแม้รู้ไม่รอด หวั...,เอ็มมา ลี คุณแม่ชาวอังกฤษ วัย 32 ปี ตัดสินใจอุ...,2015-12-04 03:11:37,foreign,https://www.matichon.co.th/foreign/news_268,268
4,"""ผบ.สส.""นำทหารรักษาพระองค์ สวนสนามเทิดพระเกียร...",เมื่อวันที่ 3 ธ.ค. ที่มณฑลพิธีท้องสนามหลวง พล....,2015-12-04 03:38:05,politics,https://www.matichon.co.th/politics/news_295,295
...,...,...,...,...,...,...
16321,'ชาคริต-เรย์' เพื่อนซี้พ่อลูกอ่อนรวมก๊วน งานนี...,เป็นเพื่อนที่สนิทกันมาแต่ไหนแต่ไร แถมพอมีลูก ก...,2018-07-05 11:43:53,entertainment,https://www.matichon.co.th/entertainment/news_...,1029324
16322,สงขลาจัดงานสมโภช 176 ปี ศาลเจ้าพ่อหลักเมือง,วันที่ 5 ก.ค. นายสมศักดิ์ ตันติเศรณี นายก ทน.ส...,2018-07-05 10:59:53,region,https://www.matichon.co.th/region/news_1029328,1029328
16323,'สมคิด' ลั่น ไม่ใช่แค่สามมิตร แต่มีมวลหมู่มหาม...,"""สมคิด"" ลั่นไม่ใช่แค่สามมิตร แต่มีมวลหมู่มหามิ...",2018-07-05 12:09:08,politics,https://www.matichon.co.th/politics/news_1029410,1029410
16324,ม.รามฯปฐมนิเทศ นศ.ใหม่ ตั้งใจศึกษาให้สำเร็จ-มี...,ผู้ช่วยศาสตราจารย์วุฒิศักดิ์ ลาภเจริญทรัพย์ อ...,2018-07-05 13:45:34,publicize,https://www.matichon.co.th/publicize/news_1029659,1029659


In [69]:
df = df[df['category']=='politics']
df

,headline,article,date,category,url,id
0,เปิดใจชัดๆ กับ'ชูวิทย์' สบู่ ขัน ต้องเตรียมไว้...,นายชูวิทย์ กมลวิศิษฎ์ หัวหน้าพรรครักประเทศไทย ...,2015-08-16 10:58:27,politics,https://www.matichon.co.th/politics/news_104,104
4,"""ผบ.สส.""นำทหารรักษาพระองค์ สวนสนามเทิดพระเกียร...",เมื่อวันที่ 3 ธ.ค. ที่มณฑลพิธีท้องสนามหลวง พล....,2015-12-04 03:38:05,politics,https://www.matichon.co.th/politics/news_295,295
5,"""ไก่อู""ชี้ ตู่-เต้น ไม่ได้มีหน้าที่ตรวจสอบทุจร...","""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",2015-12-04 04:10:49,politics,https://www.matichon.co.th/politics/news_329,329
9,พท.จัดเต็ม ออกแถลงการณ์ 3ข้อปมราชภักดิ์ จี้ รี...,เมื่อวันที่ 1 ธันวาคม ผู้สื่อข่าวรายงานว่า พรร...,2015-12-02 17:44:33,politics,https://www.matichon.co.th/politics/news_568,568
13,"The China Card : เมื่อไทยเล่น ""ไพ่จีน"" โดย สุร...",ยุทธบทความ\nมติชนสุดสัปดาห์ 11-17 ธันวาคม 2558...,2015-12-16 05:58:32,politics,https://www.matichon.co.th/politics/news_630,630
...,...,...,...,...,...,...
16299,"พระราชกระแสขอบใจ ชื่นชม ช่วย""13 หมูป่า"" ทรงรับ...",พล.ท.สรรเสริญ แก้วกำเนิด โฆษกประจำสำนักนายกรัฐ...,2018-07-03 21:32:47,politics,https://www.matichon.co.th/politics/news_1027414,1027414
16301,"09.00 INDEX กระบวนการ""ภาคเสธ""จาก""คสช."" กรณี ส...","เหมือนกับการแถลงในกรณี""กลุ่มสามมิตร"" กับ""พรรคพ...",2018-07-04 08:36:45,politics,https://www.matichon.co.th/politics/news_1027697,1027697
16310,'ผบ.ทบ.' ส่งทีมนักดำน้ำ 'มนุษย์กบ' 31 นาย จาก ...,"""ผบ.ทบ."" ส่งทีมนักดำน้ำ ""มนุษย์กบ"" 31 นาย จาก ...",2018-07-04 16:29:50,politics,https://www.matichon.co.th/politics/news_1028540,1028540
16323,'สมคิด' ลั่น ไม่ใช่แค่สามมิตร แต่มีมวลหมู่มหาม...,"""สมคิด"" ลั่นไม่ใช่แค่สามมิตร แต่มีมวลหมู่มหามิ...",2018-07-05 12:09:08,politics,https://www.matichon.co.th/politics/news_1029410,1029410


# tokenize

In [82]:
## function for tokenize
## remove stopwords
STOPWORDS = thai_stopwords() 

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token.strip('\'').strip('\"') for token in tokens] # remove quotations before/after token
    tokens = [token for token in tokens if token not in STOPWORDS] # remove stopwords
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-๙0-9][A-Za-zก-๙0-9\.\-]+', token)] # token pattern : only words/digits
    return tokens

tokenize("เปิดใจชัดๆ กับ'ชูวิทย์' สบู่ ขัน ต้องเตรียมไว้! ในคุกไม่มี 7-11")

['เปิดใจ', 'ชัด', 'ชู', 'วิทย์', 'สบู่', 'ขัน', 'เตรียม', 'คุก', '11']

In [83]:
df['tokens'] = df['article'].apply(tokenize)

/var/folders/0l/v6xrjrh90pl23576c_1w0hd40000gn/T/ipykernel_11499/480646713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['article'].apply(tokenize)


# create corpus

In [84]:
## create vocab <> ID dictionary
dictionary = Dictionary(df['tokens']) 

## create bag of words of each document
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# model

In [85]:
num_topics = 6

## instantiation
lda = LdaModel(corpus, num_topics=num_topics)

# show keywords in each topic

- `lda.get_topic_terms(topic_no)` gives list of `(word_index, probability of being the topic)`
- must `dictionary[word_index]` to get the word

In [86]:
## show top 20 words of topic 1
topic_no = 1

topic_df = pd.DataFrame(columns=['word','probability'])
for index, prob in lda.get_topic_terms(topic_no, topn=20):
    topic_df.loc[len(topic_df)] = [dictionary[index], round(prob, 3)]

topic_df

,word,probability
0,ประชาชน,0.012
1,เรื่อง,0.010
2,วันที่,0.009
3,คน,0.008
4,ทำ,0.007
5,รัฐธรรมนูญ,0.006
6,กฎหมาย,0.006
7,พล,0.006
8,สนช,0.005
9,เวลา,0.005


In [87]:
## show top 20 words of all topics
for topic_no in range(num_topics):
    print(f"---------- TOPIC {topic_no} ----------")
    top20 = [dictionary[index] for index, prob in lda.get_topic_terms(topic_no, topn=20)]
    print(' | '.join(top20), '\n')

---------- TOPIC 0 ----------
ประเทศ | ไทย | วันที่ | อ. | พล | เรื่อง | พื้นที่ | รัฐบาล | โครงการ | ปี | ประชาชน | นายกรัฐมนตรี | คน | ทำ | บาท | ประยุทธ์ | กล่าวว่า | คสช. | คณะ | การพัฒนา 

---------- TOPIC 1 ----------
ประชาชน | เรื่อง | วันที่ | คน | ทำ | รัฐธรรมนูญ | กฎหมาย | พล | สนช | เวลา | กล่าวว่า | นายกรัฐมนตรี | การเลือกตั้ง | ที่จะ | ธ. | กกต. | รัฐบาล | อ. | คณะกรรมการ | ประเทศ 

---------- TOPIC 2 ----------
พรรค | คน | คสช. | รัฐบาล | เรื่อง | พรรคการเมือง | ประชาชน | ทำ | พล | อ. | การเลือกตั้ง | ผม | นายกฯ | ไทย | วันที่ | ปี | ประยุทธ์ | การเมือง | เลือกตั้ง | ประเทศ 

---------- TOPIC 3 ----------
คน | วันที่ | เรื่อง | อ. | ปี | กกต. | พล | ประเทศ | ประชาชน | คสช. | ทำ | ชัย | รัฐบาล | ไทย | คณะกรรมการ | กฎหมาย | น. | รัฐมนตรี | เวลา | สนช 

---------- TOPIC 4 ----------
ประชาชน | พล | วันที่ | อ. | จังหวัด | เรื่อง | ที่จะ | ไทย | รัฐบาล | คน | ตรวจสอบ | พื้นที่ | ปี | ทำ | กล่าวว่า | ประเทศ | คสช. | ธ. | ขสมก. | ดำเนินการ 

---------- TOPIC 5 ----------
อ. | ไท